In [1]:
import torch 
from torch import nn ,optim
from torch.nn import functional as F
from torchvision import datasets, transforms
%matplotlib inline
import matplotlib.pyplot as plt

In [4]:

#Normalizing the data and transforming to tensor
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,),(0.5,)),
                                ])
#Downloading the train data and test data
trainset = datasets.EMNIST("~/.pytorch/EMNIST_data/",download =True,train=True,transform=transform,split="byclass")
testset = datasets.EMNIST("~/.pytorch/EMNIST_data/",download=True,train=False,transform=transform,split = "byclass")



In [5]:
print(trainset.classes)

['d', 'O', 'g', 'C', '0', 'f', 'n', 'J', '4', '9', 'U', 'R', 'B', 'c', 'w', 'Q', 'K', '7', '5', 'L', 'W', 's', 'H', 'Y', 'E', 'M', 'z', 'x', 'F', 'k', 'G', '6', 'N', '2', 'D', 'P', 'i', 'o', 'y', 'I', 'q', 'A', '1', 'r', 'b', 'Z', 'a', 'h', 'm', 'v', '8', 'S', 'V', 'X', 'u', '3', 't', 'T', 'l', 'j', 'e', 'p']


In [6]:
len(trainset.classes)

62

In [7]:
trainset

Dataset EMNIST
    Number of datapoints: 697932
    Root location: /root/.pytorch/EMNIST_data/
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.5,), std=(0.5,))
           )

In [9]:
class Classifier(nn.Module):
  def __init__(self):
    super().__init__() # it track architecture by nn.Module class
    self.conv1 = nn.Conv2d(in_channels=1, out_channels=10, kernel_size=3  )
    self.conv2 = nn.Conv2d(in_channels=10, out_channels=10, kernel_size=3  )
    self.pool = nn.MaxPool2d(2, 2)
    self.conv3 = nn.Conv2d(in_channels=10, out_channels=20, kernel_size=3  )
    self.conv4 = nn.Conv2d(in_channels=20, out_channels=20, kernel_size=3  )
    self.conv5 = nn.Conv2d(in_channels=20, out_channels=30, kernel_size=3  )
    self.conv6 = nn.Conv2d(in_channels=30, out_channels=62, kernel_size=3  )
    self.gap = nn.AvgPool2d(kernel_size=4)
  def forward(self,x):
    x = self.conv1(x)
    x = F.relu(self.conv2(x))
    x = self.pool(x)
    x = F.relu(self.conv3(x))
    x = F.relu(self.conv4(x))
    x = F.relu(self.conv5(x))
    x = F.relu(self.conv6(x))
    x = self.gap(x)
    x = F.log_softmax(x.view(-1,62), dim=1)
    return x

from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Classifier().to(device)
summary(model, input_size=(1, 28, 28))

# below layers are explained nothing but Conv2d-1  is [-1, 10, 26, 26] is-1 is nothing but batchsize 10 is neurons or channels or kernals , 26, 26 nothing but receptive fields
# 100 means 100 trainable parameters like that so on......

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 10, 26, 26]             100
            Conv2d-2           [-1, 10, 24, 24]             910
         MaxPool2d-3           [-1, 10, 12, 12]               0
            Conv2d-4           [-1, 20, 10, 10]           1,820
            Conv2d-5             [-1, 20, 8, 8]           3,620
            Conv2d-6             [-1, 30, 6, 6]           5,430
            Conv2d-7             [-1, 62, 4, 4]          16,802
         AvgPool2d-8             [-1, 62, 1, 1]               0
Total params: 28,682
Trainable params: 28,682
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.15
Params size (MB): 0.11
Estimated Total Size (MB): 0.26
----------------------------------------------------------------


In [11]:
#Loading the train data and test data
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
trainloader = torch.utils.data.DataLoader(trainset,batch_size=64,**kwargs)
testloader = torch.utils.data.DataLoader(testset,batch_size=64,**kwargs)

In [14]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.1f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [16]:
model = Classifier().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 20):
    train(model, device, trainloader, optimizer, epoch)
    test(model, device, testloader)

  0%|          | 0/10906 [00:00<?, ?it/s]


Test set: Average loss: 0.4684, Accuracy: 97431/116323 (83.8%)



  0%|          | 0/10906 [00:00<?, ?it/s]


Test set: Average loss: 0.4348, Accuracy: 98808/116323 (84.9%)



  0%|          | 0/10906 [00:00<?, ?it/s]

...


Test set: Average loss: 0.3888, Accuracy: 99811/116323 (85.8%)



  0%|          | 0/10906 [00:00<?, ?it/s]


Test set: Average loss: 0.3864, Accuracy: 99878/116323 (85.9%)



loss=0.2194817215204239 batch_id=10905: 100%|██████████| 10906/10906 [02:59<00:00, 60.64it/s]



Test set: Average loss: 0.3873, Accuracy: 99848/116323 (85.8%)

